# **Fracture energy for mode 1 and mode 2**

In [1]:
%matplotlib widget
import bmcs_utils
from ibvpy.tfunction import TimeFunction, TFSelector, TFCyclicSin
from bmcs_matmod.slide.slide_explorer import SlideExplorer
import bmcs_utils.api as bu

## 1 Pull off loading

In [2]:
material_params = dict(
    E_T=1, gamma_T=0, K_T=0, S_T=5, c_T=1, bartau=3, 
    E_N=1, S_N=1, c_N = 1, m = 0.01, f_t=3, f_c=20, f_c0=10, eta=0.0 # 0.5
)

In [3]:
se_N = SlideExplorer(n_steps=2000, k_max=50)
se_N.slide_model.trait_set(**material_params)
se_N.trait_set(s_x_1 = 0, s_y_1 = 0, w_1 = 50);
se_N.run()
se_N.energy_dissipation.show_iter = False
se_N.interact()

In [4]:
diss_N = se_N.energy_dissipation.mechanisms.damage_diss[-1], se_N.energy_dissipation.mechanisms.plastic_diss[-1]
diss_N

(4.479437182100187, 3.4215093777180186)

# Sliding loading

In [5]:
se_T = SlideExplorer(n_steps=2000, k_max=50)
se_T.slide_model.trait_set(**material_params)
se_T.trait_set(s_x_1 = 50, s_y_1 = 0, w_1 = 0);
se_T.run()
se_T.energy_dissipation.show_iter = False
se_T.interact()

In [6]:
diss_T = se_T.energy_dissipation.mechanisms.damage_diss[-1], se_T.energy_dissipation.mechanisms.plastic_diss[-1]
diss_T

(4.398257395714287, 12.338747930789028)

# Pull off - Parametric study of energy dissipation

Evaluate energy required to produce a stress-free interface in the normal direction

In [10]:
import numpy as np
se_N = SlideExplorer(n_steps=2000, k_max=500)
se_N.slide_model.trait_set(**material_params)
se_N.energy_dissipation.show_iter = False
S_N_range = np.linspace(-0.001, 1, 5)
c_N_range = np.linspace(-0.001, 1, 5)
S_N_grid, c_N_grid = np.meshgrid(S_N_range, c_N_range)
damage_diss_grid = np.zeros_like(S_N_grid)
plastic_diss_grid = np.zeros_like(S_N_grid)
resid_stress_grid = np.zeros_like(S_N_grid)
for i, S_N in enumerate(S_N_range):
    for j, c_N in enumerate(c_N_range):
        print('evaluating', S_N, c_N)
        se_N.slide_model.S_N = S_N
        se_N.slide_model.c_N = c_N
        se_N.reset()
        se_N.trait_set(s_x_1 = 0, s_y_1 = 0, w_1 = 50);
        se_N.run()
        damage_diss_grid[i,j] = se_N.energy_dissipation.mechanisms.damage_diss[-1]
        plastic_diss_grid[i,j] = se_N.energy_dissipation.mechanisms.plastic_diss[-1]
        resid_stress_grid[i,j] = se_N.Sig_arr[-1,0] / np.max(se_N.Sig_arr[:,0])

evaluating -0.001 -0.001


ConvergenceError: no convergence for step for state [1e-09, 1e-09, 3.025000001]

In [ ]:
import matplotlib.pylab as plt
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(S_N_grid, c_N_grid, damage_diss_grid, color='gray', label='damage', alpha=0.4)
ax.scatter(S_N_grid, c_N_grid, damage_diss_grid, marker='H')
ax.plot_surface(S_N_grid, c_N_grid, plastic_diss_grid, color='red', label='plastic', alpha=0.5)
ax.scatter(S_N_grid, c_N_grid, plastic_diss_grid, marker='H')
ax.set_xlabel('S_N'); ax.set_ylabel('c_N');

In [9]:
resid_stress_grid

array([[7.14738966e-11, 2.30803786e-10, 5.31162780e-10, 9.93673213e-10,
        1.62149891e-09],
       [7.86455754e-11, 2.49230544e-10, 5.66373404e-10, 1.04998662e-09,
        1.70174849e-09],
       [8.61434474e-11, 2.68149504e-10, 6.02087396e-10, 1.10660844e-09,
        1.78190891e-09],
       [9.39653199e-11, 2.87548480e-10, 6.38285691e-10, 1.16352264e-09,
        1.86198023e-09],
       [1.02109012e-10, 3.07415969e-10, 6.74950576e-10, 1.22071451e-09,
        1.94196252e-09]])

In [ ]:
se_N.interact()

In [10]:
se_N.interact()